# Python执行过程

## 入口 
Programs/python.c
```
int
main(int argc, char **argv)
{
    // unix平台是_Py_UnixMain，windows平台是Py_Main
    return _Py_UnixMain(argc, argv);
}
```

然后为Modules/main.c
```
int
_Py_UnixMain(int argc, char **argv)
{
    _PyMain pymain = _PyMain_INIT;
    pymain.use_bytes_argv = 1;
    pymain.argc = argc;
    pymain.bytes_argv = argv;

    return pymain_main(&pymain);
}
```

```
static int
pymain_main(_PyMain *pymain)
{
    // 初始化
    pymain_init(pymain);

    int res = pymain_cmdline(pymain);
    if (res < 0) {
        _Py_FatalInitError(pymain->err);
    }
    if (res == 1) {
        goto done;
    }

    pymain_init_stdio(pymain);

    pymain->err = _Py_InitializeCore(&pymain->config);
    if (_Py_INIT_FAILED(pymain->err)) {
        _Py_FatalInitError(pymain->err);
    }

    if (pymain_init_python_main(pymain) < 0) {
        _Py_FatalInitError(pymain->err);
    }

    if (pymain_init_sys_path(pymain) < 0) {
        _Py_FatalInitError(pymain->err);
    }

    pymain_run_python(pymain);

    if (Py_FinalizeEx() < 0) {
        pymain->status = 120;
    }

done:
    // 退出清理
    pymain_free(pymain);

    return pymain->status;
}
```

```
static void
pymain_run_python(_PyMain *pymain)
{
    PyCompilerFlags cf = {.cf_flags = 0};

    pymain_header(pymain);
    pymain_import_readline(pymain);
    
    // 执行模式选择
    if (pymain->command) {
        // 命令行模式 -c
        pymain->status = pymain_run_command(pymain->command, &cf);
    }
    else if (pymain->module) {
        // 模块模式 -m
        pymain->status = (pymain_run_module(pymain->module, 1) != 0);
    }
    else {
        // 入口文件模式
        pymain_run_filename(pymain, &cf);
    }

    pymain_repl(pymain, &cf);
}
```


## 初始化
除内置类型外，最重要的是创建buildins、sys模块，并初始化sys.modules，sys.path等运行所需的环境配置。  
Python/pylifecycle.c  
```
_PyInitError
_Py_InitializeCore(const _PyCoreConfig *core_config)
{
    // 创建解释器状态实例
    interp = PyInterpreterState_New();
    if (interp == NULL) {
        return _Py_INIT_ERR("can't make main interpreter");
    }
    // 创建主线程状态实例
    tstate = PyThreadState_New(interp);
    if (tstate == NULL)
        return _Py_INIT_ERR("can't make first thread");
    (void) PyThreadState_Swap(tstate);

    _PyEval_FiniThreads();

    /* Auto-thread-state API */
    _PyGILState_Init(interp, tstate);

    /* Create the GIL */
    PyEval_InitThreads();
    
    // 创建并初始化内置类型
    _Py_ReadyTypes();
    
    // 初始化带有对象缓存的内置类型
    if (!_PyLong_Init())
        return _Py_INIT_ERR("can't init longs");

    if (!PyByteArray_Init())
        return _Py_INIT_ERR("can't init bytearray");

    if (!_PyFloat_Init())
        return _Py_INIT_ERR("can't init float");

    PyObject *modules = PyDict_New();
    if (modules == NULL)
        return _Py_INIT_ERR("can't make modules dictionary");
    // 创建sys.modules,存储运行期被导入的模块
    interp->modules = modules;
    
    // 初始化sys模块
    err = _PySys_BeginInit(&sysmod);
    if (_Py_INIT_FAILED(err)) {
        return err;
    }
    interp->sysdict = PyModule_GetDict(sysmod);
    if (interp->sysdict == NULL) {
        return _Py_INIT_ERR("can't initialize sys dict");
    }

    Py_INCREF(interp->sysdict);
    PyDict_SetItemString(interp->sysdict, "modules", modules);
    _PyImport_FixupBuiltin(sysmod, "sys", modules);

    /* Init Unicode implementation; relies on the codec registry */
    if (_PyUnicode_Init() < 0)
        return _Py_INIT_ERR("can't initialize unicode");

    if (_PyStructSequence_Init() < 0)
        return _Py_INIT_ERR("can't initialize structseq");
    // 初始化 __buildin__ 模块
    bimod = _PyBuiltin_Init();
    if (bimod == NULL)
        return _Py_INIT_ERR("can't initialize builtins modules");
    _PyImport_FixupBuiltin(bimod, "builtins", modules);
    interp->builtins = PyModule_GetDict(bimod);
    if (interp->builtins == NULL)
        return _Py_INIT_ERR("can't initialize builtins dict");
    Py_INCREF(interp->builtins);

    // 初始化内置异常类型
    _PyExc_Init(bimod);

    /* Set up a preliminary stderr printer until we have enough
       infrastructure for the io module in place. */
    pstderr = PyFile_NewStdPrinter(fileno(stderr));
    if (pstderr == NULL)
        return _Py_INIT_ERR("can't set preliminary stderr");
    _PySys_SetObjectId(&PyId_stderr, pstderr);
    PySys_SetObject("__stderr__", pstderr);
    Py_DECREF(pstderr);
    
    // 初始化导入机制
    err = _PyImport_Init(interp);
    if (_Py_INIT_FAILED(err)) {
        return err;
    }

    err = _PyImportHooks_Init();
    if (_Py_INIT_FAILED(err)) {
        return err;
    }

    /* Initialize _warnings. */
    if (_PyWarnings_Init() == NULL) {
        return _Py_INIT_ERR("can't initialize warnings");
    }

    if (!_PyContext_Init())
        return _Py_INIT_ERR("can't init context");

    /* This call sets up builtin and frozen import support */
    if (!interp->core_config._disable_importlib) {
        err = initimport(interp, sysmod);
        if (_Py_INIT_FAILED(err)) {
            return err;
        }
    }

    /* Only when we get here is the runtime core fully initialized */
    _PyRuntime.core_initialized = 1;
    return _Py_INIT_OK();
}
```

## 执行
完成初始化后，根据选定的执行模式进入对应的执行方式，这里列出源文件执行方式。  
Modules/main.c
```
static int
pymain_run_file(FILE *fp, const wchar_t *filename, PyCompilerFlags *p_cf)
{
    ...

    run = PyRun_AnyFileExFlags(fp, filename_str, filename != NULL, p_cf);
    Py_XDECREF(bytes);
    return run != 0;
}
```

Python/pythonrun.c
```
/* Parse input from a file and execute it */
int
PyRun_AnyFileExFlags(FILE *fp, const char *filename, int closeit,
                     PyCompilerFlags *flags)
{
    ...
    else
        return PyRun_SimpleFileExFlags(fp, filename, closeit, flags);
}
```

```
int
PyRun_SimpleFileExFlags(FILE *fp, const char *filename, int closeit,
                        PyCompilerFlags *flags)
{
    PyObject *m, *d, *v;
    const char *ext;
    int set_file_name = 0, ret = -1;
    size_t len;
    
    // 获取__main__.__dict__，添加__file__ 信息
    m = PyImport_AddModule("__main__");
    if (m == NULL)
        return -1;
    Py_INCREF(m);
    d = PyModule_GetDict(m);
    if (PyDict_GetItemString(d, "__file__") == NULL) {
        PyObject *f;
        f = PyUnicode_DecodeFSDefault(filename);
        if (f == NULL)
            goto done;
        if (PyDict_SetItemString(d, "__file__", f) < 0) {
            Py_DECREF(f);
            goto done;
        }
        if (PyDict_SetItemString(d, "__cached__", Py_None) < 0) {
            Py_DECREF(f);
            goto done;
        }
        set_file_name = 1;
        Py_DECREF(f);
    }
    len = strlen(filename);
    ext = filename + len - (len > 4 ? 4 : 0);
    
    // 运行入口文件（检查pyc缓存），将__main__.__dict__做为名字空间传入
    if (maybe_pyc_file(fp, filename, ext, closeit)) {
        FILE *pyc_fp;
        /* Try to run a pyc file. First, re-open in binary */
        if (closeit)
            fclose(fp);
        if ((pyc_fp = _Py_fopen(filename, "rb")) == NULL) {
            fprintf(stderr, "python: Can't reopen .pyc file\n");
            goto done;
        }

        if (set_main_loader(d, filename, "SourcelessFileLoader") < 0) {
            fprintf(stderr, "python: failed to set __main__.__loader__\n");
            ret = -1;
            fclose(pyc_fp);
            goto done;
        }
        // 运行缓存文件
        v = run_pyc_file(pyc_fp, filename, d, d, flags);
        fclose(pyc_fp);
    } else {
        /* When running from stdin, leave __main__.__loader__ alone */
        if (strcmp(filename, "<stdin>") != 0 &&
            set_main_loader(d, filename, "SourceFileLoader") < 0) {
            fprintf(stderr, "python: failed to set __main__.__loader__\n");
            ret = -1;
            goto done;
        }
        // 运行py文件
        v = PyRun_FileExFlags(fp, filename, Py_file_input, d, d,
                              closeit, flags);
    }
    flush_io();
    if (v == NULL) {
        PyErr_Print();
        goto done;
    }
    Py_DECREF(v);
    ret = 0;
  done:
    if (set_file_name && PyDict_DelItemString(d, "__file__"))
        PyErr_Clear();
    Py_DECREF(m);
    return ret;
}
```

```
PyObject *
PyRun_FileExFlags(FILE *fp, const char *filename_str, int start, PyObject *globals,
                  PyObject *locals, int closeit, PyCompilerFlags *flags)
{
    PyObject *ret = NULL;
    mod_ty mod;
    PyArena *arena = NULL;
    PyObject *filename;

    filename = PyUnicode_DecodeFSDefault(filename_str);
    if (filename == NULL)
        goto exit;

    arena = PyArena_New();
    if (arena == NULL)
        goto exit;
    // 编译源文件
    mod = PyParser_ASTFromFileObject(fp, filename, NULL, start, 0, 0,
                                     flags, NULL, arena);
    if (closeit)
        fclose(fp);
    if (mod == NULL) {
        goto exit;
    }
    // 字节码执行流程
    ret = run_mod(mod, filename, globals, locals, flags, arena);

exit:
    Py_XDECREF(filename);
    if (arena != NULL)
        PyArena_Free(arena);
    return ret;
}
```

```
static PyObject *
run_mod(mod_ty mod, PyObject *filename, PyObject *globals, PyObject *locals,
            PyCompilerFlags *flags, PyArena *arena)
{
    co = PyAST_CompileObject(mod, filename, flags, -1, arena);
    v = PyEval_EvalCode((PyObject*)co, globals, locals);
}
```

Python/ceval.c
```
PyObject *
PyEval_EvalCode(PyObject *co, PyObject *globals, PyObject *locals)
{
    return PyEval_EvalCodeEx(co,
                      globals, locals,
                      (PyObject **)NULL, 0,
                      (PyObject **)NULL, 0,
                      (PyObject **)NULL, 0,
                      NULL, NULL);
}
```

```
PyObject *
PyEval_EvalCodeEx(PyObject *_co, PyObject *globals, PyObject *locals,
                  PyObject *const *args, int argcount,
                  PyObject *const *kws, int kwcount,
                  PyObject *const *defs, int defcount,
                  PyObject *kwdefs, PyObject *closure)
{
    return _PyEval_EvalCodeWithName(_co, globals, locals,
                                    args, argcount,
                                    kws, kws != NULL ? kws + 1 : NULL,
                                    kwcount, 2,
                                    defs, defcount,
                                    kwdefs, closure,
                                    NULL, NULL);
}
```

```
PyObject *
_PyEval_EvalCodeWithName(PyObject *_co, PyObject *globals, PyObject *locals,
           PyObject *const *args, Py_ssize_t argcount,
           PyObject *const *kwnames, PyObject *const *kwargs,
           Py_ssize_t kwcount, int kwstep,
           PyObject *const *defs, Py_ssize_t defcount,
           PyObject *kwdefs, PyObject *closure,
           PyObject *name, PyObject *qualname)
{

    // 创建栈帧
    tstate = PyThreadState_GET();
    assert(tstate != NULL);
    f = _PyFrame_New_NoTrack(tstate, co, globals, locals);
    ···
    填充参数、自由变量（闭包）等数据
    ···
    retval = PyEval_EvalFrameEx(f,0);
}
```

```
PyObject *
PyEval_EvalFrameEx(PyFrameObject *f, int throwflag)
{
    PyThreadState *tstate = PyThreadState_GET();
    return tstate->interp->eval_frame(f, throwflag);
    // eval_frame : 初始化在 PyEval_EvalFrameDefault()
}
```

```
PyObject* _Py_HOT_FUNCTION
_PyEval_EvalFrameDefault(PyFrameObject *f, int throwflag)
{
    ···
    // 指令参数所需的相关名字列表
    co = f->f_code;
    names = co->co_names;
    consts = co->co_consts;
    fastlocals = f->f_localsplus;
    freevars = f->f_localsplus + co->co_nlocals;
    
    // 类似于SP、PC寄存器，下一条指令及栈帧顶位置
    first_instr = (_Py_CODEUNIT *) PyBytes_AS_STRING(co->co_code);
    next_instr = first_instr;
    if (f->f_lasti >= 0) {
        assert(f->f_lasti % sizeof(_Py_CODEUNIT) == 0);
        next_instr += f->f_lasti / sizeof(_Py_CODEUNIT) + 1;
    }
    stack_pointer = f->f_stacktop;
    assert(stack_pointer != NULL);
    f->f_stacktop = NULL;       /* remains NULL unless yield suspends frame */
    f->f_executing = 1;
    
    // 解释循环
    main_loop:
    for (;;) {
        // 检查并处理GIL
        if (_Py_atomic_load_relaxed(&_PyRuntime.ceval.eval_breaker)) {
            ···
        }

    fast_next_opcode:
        f->f_lasti = INSTR_OFFSET();
        /* Extract opcode and argument */
        // 下一条指令和参数
        NEXTOPARG();
    dispatch_opcode:
        
        // 指令执行，每条指令由C实现具体过程
        switch (opcode) {

        TARGET(NOP)
            FAST_DISPATCH();

        TARGET(LOAD_FAST) {
            PyObject *value = GETLOCAL(oparg);
            if (value == NULL) {
                format_exc_check_arg(PyExc_UnboundLocalError,
                                     UNBOUNDLOCAL_ERROR_MSG,
                                     PyTuple_GetItem(co->co_varnames, oparg));
                goto error;
            }
            Py_INCREF(value);
            PUSH(value);
            FAST_DISPATCH();
        }

        PREDICTED(LOAD_CONST);
        TARGET(LOAD_CONST) {
            PyObject *value = GETITEM(consts, oparg);
            Py_INCREF(value);
            PUSH(value);
            FAST_DISPATCH();
        }
        
        ···
     }
     exit_eval_frame:
        Py_LeaveRecursiveCall();
        f->f_executing = 0;
        tstate->frame = f->f_back;
    return _Py_CheckFunctionResult(NULL, retval, "PyEval_EvalFrameEx");
}
        
```
用户栈内存按地址从低向高分配，每次执行递增指令计数器。
```
#define NEXTOPARG()  do { \
        _Py_CODEUNIT word = *next_instr; \
        opcode = _Py_OPCODE(word); \
        oparg = _Py_OPARG(word); \
        next_instr++; \
    } while (0)
#define EMPTY()           (STACK_LEVEL() == 0)
#define TOP()             (stack_pointer[-1])
#define SECOND()          (stack_pointer[-2])
#define THIRD()           (stack_pointer[-3])
#define FOURTH()          (stack_pointer[-4])
#define PEEK(n)           (stack_pointer[-(n)])
#define SET_TOP(v)        (stack_pointer[-1] = (v))
#define SET_SECOND(v)     (stack_pointer[-2] = (v))
#define SET_THIRD(v)      (stack_pointer[-3] = (v))
#define SET_FOURTH(v)     (stack_pointer[-4] = (v))
#define SET_VALUE(n, v)   (stack_pointer[-(n)] = (v))
#define BASIC_STACKADJ(n) (stack_pointer += n)
#define BASIC_PUSH(v)     (*stack_pointer++ = (v)) // 地址从低向高进行
#define BASIC_POP()       (*--stack_pointer)
```

## 终止
在结束前，会完成一系列清理操作。  
Python/pylifecycle.c
```
int
Py_FinalizeEx(void)
{
    PyInterpreterState *interp;
    PyThreadState *tstate;
    int status = 0;

    if (!_PyRuntime.initialized)
        return status;
    
    // 等待前台线程结束
    wait_for_thread_shutdown();

    /* Get current thread state and interpreter pointer */
    tstate = PyThreadState_GET();
    interp = tstate->interp;
    
    // 调用 atexit 注册的退出函数
    call_py_exitfuncs(interp);

    /* Flush sys.stdout and sys.stderr */
    if (flush_std_files() < 0) {
        status = -1;
    }

    /* Disable signal handling */
    PyOS_FiniInterrupts();

    // 垃圾回收，执行析构方法
    _PyGC_CollectIfEnabled();

    // 释放导入的模块
    PyImport_Cleanup();

    // 执行相关结束函数
    /* Disable tracemalloc after all Python objects have been destroyed,
       so it is possible to use tracemalloc in objects destructor. */
    _PyTraceMalloc_Fini();

    /* Destroy the database used by _PyImport_{Fixup,Find}Extension */
    _PyImport_Fini();

    /* Cleanup typeobject.c's internal caches. */
    _PyType_Fini();

    /* unload faulthandler module */
    _PyFaulthandler_Fini();


    _PyExc_Fini();

    // 执行内置类型的结束函数
    PyMethod_Fini();
    PyFrame_Fini();
    PyCFunction_Fini();
    PyTuple_Fini();
    PyList_Fini();
    PySet_Fini();
    PyBytes_Fini();
    PyByteArray_Fini();
    PyLong_Fini();
    PyFloat_Fini();
    PyDict_Fini();
    PySlice_Fini();
    _PyGC_Fini();
    _Py_HashRandomization_Fini();
    _PyArg_Fini();
    PyAsyncGen_Fini();
    _PyContext_Fini();

    /* Cleanup Unicode implementation */
    _PyUnicode_Fini();

    /* reset file system default encoding */
    if (!Py_HasFileSystemDefaultEncoding && Py_FileSystemDefaultEncoding) {
        PyMem_RawFree((char*)Py_FileSystemDefaultEncoding);
        Py_FileSystemDefaultEncoding = NULL;
    }

    /* XXX Still allocated:
       - various static ad-hoc pointers to interned strings
       - int and float free list blocks
       - whatever various modules and libraries allocate
    */

    PyGrammar_RemoveAccelerators(&_PyParser_Grammar);

    /* Cleanup auto-thread-state */
    _PyGILState_Fini();

    /* Delete current thread. After this, many C API calls become crashy. */
    PyThreadState_Swap(NULL);
    // 清理解释器和主线程状态
    PyInterpreterState_Delete(interp);


    call_ll_exitfuncs();

    _PyRuntime_Finalize();
    return status;
}
```